In [18]:
%load_ext autoreload
%autoreload 2
%cd C:\MAD4AG
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\MAD4AG


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from collections import Counter


import warnings

warnings.filterwarnings('ignore')

In [20]:
df_survey_act= pd.read_pickle(f'./dbs/survey_act_weekday.pkl')
df_survey_dist= pd.read_pickle(f'./dbs/intermediate/df_survey_dist.pkl')



## filter ppl by
- participating work activity
- county
- urban/rural
- commute distance (is it above or below the median in the survey)
- avg trip distance

In [21]:
# create tuple showing all activities of an individual

df_survey_ppl= df_survey_act.groupby('sub_id')['purpose'].apply(tuple).reset_index(name='act_seq')

df_survey_ppl.act_seq.value_counts(normalize=True)

(Home, Work, Home)                                                                         0.187333
(Home, Other, Home)                                                                        0.186245
(Home, Home)                                                                               0.060862
(Home, Other, Home, Other, Home)                                                           0.048255
(Home, Other, Other, Home)                                                                 0.044596
                                                                                             ...   
(Home, Other, Other, Other, Work, Other, Other, Other, Work, Other, Other, Other, Home)    0.000049
(Home, Other, Work, Other, Work, Other, Other, Other, Home, Other, Home)                   0.000049
(Home, Other, Other, Work, Other, Work, Other, Home)                                       0.000049
(Home, Other, Home, Other, Other, Work, Home)                                              0.000049


In [22]:
# find the unique activities from the activity tuple


def unique_activities(data):
    act_tuple = data['act_seq']
    data['act_uniq'] = tuple(Counter(act_tuple).keys())
    # this will take only unique numbers from the tuple
    return data


In [23]:
tqdm.pandas()
df_survey_ppl= df_survey_ppl.apply(lambda row: unique_activities(row) , axis=1 )

In [24]:
df_survey_ppl.act_uniq.value_counts()

(Home, Other)                  8488
(Home, Work)                   4499
(Home, Work, Other)            3381
(Home,)                        1568
(Home, Other, Work)            1387
(Home, School)                  392
(Home, School, Other)           270
(Work, Home)                     72
(Home, Other, School)            66
(Work, Home, Other)              56
(Work, Other, Home)              18
(Home, School, Work)             13
(Home, Work, School)              7
(Home, Work, School, Other)       3
(Home, Other, Work, School)       2
(Home, School, Other, Work)       2
(Home, Other, School, Work)       1
(Home, School, Work, Other)       1
Name: act_uniq, dtype: int64

In [25]:
# remove individuals having only home activity
df_survey_ppl = df_survey_ppl[df_survey_ppl.act_uniq != ('Home',)]

In [26]:
# bring the distance information

df_survey_ppl = pd.merge(df_survey_ppl, df_survey_dist, on='sub_id', how='left')

df_survey_ppl = df_survey_ppl[df_survey_ppl.avg_dist.notnull()]

df_survey_ppl['commute']= df_survey_ppl['commute'].astype(int)

In [27]:
df_survey_ppl['county']= df_survey_ppl['origin_main_deso'].str[:2]

df_survey_ppl['urban_density']= df_survey_ppl['origin_main_deso'].str[4]

In [28]:
df_survey_ppl.act_seq.value_counts(normalize=True)

(Home, Work, Home)                                                                                       0.192982
(Home, Other, Home)                                                                                      0.186933
(Home, Other, Home, Other, Home)                                                                         0.051691
(Home, Other, Other, Home)                                                                               0.050212
(Home, Work, Other, Home)                                                                                0.045910
                                                                                                           ...   
(Home, Other, Other, Other, Work, Home, Other, Work, Home)                                               0.000067
(Home, Work, Home, Work, Home, Work, Home, Work, Home)                                                   0.000067
(Home, Other, Other, Other, Other, Work, Other, Other, Other, Other, Other, Other, Home,

In [29]:
df_survey_ppl.to_pickle(r'.\dbs\intermediate\df_survey_twins.pkl')
